# Airport Security Simulation

This notebook includes:
* Using SimPy to simulate Airport Security and determine optimal number of ID Check Stations and Personal Scanners to reduce wait times to under 15 minutes
* NOTE: Arrival Time and Simulation Time have been decreased to reduce computational resources

In [1]:
import random
import simpy
import numpy as np

In [2]:
#Passengers arrive (according to poisson distribution) and go through ID Check (exponential service time)
#Passengers assigned to shortest personal check queue, personal scanner (uniformly distributed)
#Passenger leaves

In [3]:
# Set seed to reproduce results
RANDOM_SEED = 18

# Number of ID Check Stations
NUM_ID_CHECK = 4      

# Number of Personal Scanners
NUM_SCANNER = 2       

# Create a passenger based on poisson distribution (lam = 0.2) or 5 passengers/minute
ARRIVAL_TIME = 0.2

# Create time for ID Check Station based on exponential distribution (lam = 0.75)
ID_CHECK_TIME = 0.75       

# Simulation time in minutes
SIM_TIME = 240

# Create list to calculate and track wait times.
wait_times = []

In [4]:
class AirportSecurity(object):
    """An ID Check Station has a limited number of stations (``NUM_ID_CHECK``) to
    check IDs.

    Passengers have to request one of the stations. When they get one, they
    can start the ID Check process and wait for it to finish (which
    takes ``ID_CHECK_TIME`` minutes).

    """
    def __init__(self, env, num_id_check, id_check_time, num_scanner):
        self.env = env
        self.station = simpy.Resource(env, num_id_check)
        self.id_check_time = id_check_time
        self.personal_scanner = simpy.Resource(env, num_scanner)
        

    def id_check(self, passenger):
        """The ID Check process. It takes a ``passenger`` and checks their ID/Boarding Pass."""
        yield self.env.timeout(np.random.poisson(lam = ID_CHECK_TIME))
        print("%s's ID/Boarding Pass has been checked and verified." %
              passenger)
        
    def scanner(self, passenger):
        """The Personal Scanner process. It takes a ``passenger`` and the passenger goes 
        through a personal scanner"""
        yield self.env.timeout(np.random.uniform(low = 0.5, high = 1.0))
        print("%s completes personal scanning process." %
              passenger)

In [5]:
def passenger(env, name, aps):
    """The passenger process (each paseenger has a ``name``) arrives at the airport security
    (``aps``) and requests an ID Check Station, goes through a Personal Scanner, and leaves 
    the Airport Security environment.
    """
    print('%s arrives at the Airport Security at %.2f.' % (name, env.now))
    start = env.now
    with aps.station.request() as request:
        yield request

        print('%s enters ID Check Station at %.2f.' % (name, env.now))
        yield env.process(aps.id_check(name))
        
    with aps.personal_scanner.request() as request:
        print('%s enters Personal Scanner at %.2f.' % (name, env.now))
        yield env.process(aps.scanner(name))
    
    print('%s leaves Airport Security at %.2f.' % (name, env.now))
    wait_times.append(env.now - start)
    

In [6]:
def setup(env, num_id_check, id_check_time, num_scanner, arrival_time):
    """Create an airport security environment and create passengers
    approx. every ``arrival_time`` minutes."""
    # Create the airport security environment
    airport_security = AirportSecurity(env, num_id_check, id_check_time, num_scanner)
    
    # Create passengers while the simulation is running
    i = 0
    while True:
        yield env.timeout(np.random.poisson(lam=ARRIVAL_TIME))
        i += 1
        env.process(passenger(env, 'Passenger %d' % i, airport_security))

In [7]:
# Setup and start the simulation
print('Airport Security')
random.seed(RANDOM_SEED)  # Helps reproduce results

# Create an environment and start the setup process
env = simpy.Environment()
env.process(setup(env, NUM_ID_CHECK, ID_CHECK_TIME, NUM_SCANNER, ARRIVAL_TIME))

# Execute!
env.run(until=SIM_TIME)

Airport Security
Passenger 1 arrives at the Airport Security at 0.00.
Passenger 2 arrives at the Airport Security at 0.00.
Passenger 1 enters ID Check Station at 0.00.
Passenger 3 arrives at the Airport Security at 0.00.
Passenger 2 enters ID Check Station at 0.00.
Passenger 1's ID/Boarding Pass has been checked and verified.
Passenger 4 arrives at the Airport Security at 0.00.
Passenger 3 enters ID Check Station at 0.00.
Passenger 2's ID/Boarding Pass has been checked and verified.
Passenger 1 enters Personal Scanner at 0.00.
Passenger 5 arrives at the Airport Security at 0.00.
Passenger 4 enters ID Check Station at 0.00.
Passenger 3's ID/Boarding Pass has been checked and verified.
Passenger 2 enters Personal Scanner at 0.00.
Passenger 6 arrives at the Airport Security at 0.00.
Passenger 5 enters ID Check Station at 0.00.
Passenger 4's ID/Boarding Pass has been checked and verified.
Passenger 3 enters Personal Scanner at 0.00.
Passenger 7 arrives at the Airport Security at 0.00.
Pass

Passenger 238 enters ID Check Station at 44.00.
Passenger 239 enters ID Check Station at 44.00.
Passenger 240 enters ID Check Station at 44.00.
Passenger 280 arrives at the Airport Security at 44.00.
Passenger 238's ID/Boarding Pass has been checked and verified.
Passenger 239's ID/Boarding Pass has been checked and verified.
Passenger 240's ID/Boarding Pass has been checked and verified.
Passenger 281 arrives at the Airport Security at 44.00.
Passenger 238 enters Personal Scanner at 44.00.
Passenger 239 enters Personal Scanner at 44.00.
Passenger 240 enters Personal Scanner at 44.00.
Passenger 282 arrives at the Airport Security at 44.00.
Passenger 283 arrives at the Airport Security at 44.00.
Passenger 241 enters ID Check Station at 44.00.
Passenger 242 enters ID Check Station at 44.00.
Passenger 243 enters ID Check Station at 44.00.
Passenger 284 arrives at the Airport Security at 44.00.
Passenger 241's ID/Boarding Pass has been checked and verified.
Passenger 285 arrives at the Air

Passenger 402 completes personal scanning process.
Passenger 402 leaves Airport Security at 76.97.
Passenger 428 arrives at the Airport Security at 77.00.
Passenger 405's ID/Boarding Pass has been checked and verified.
Passenger 406's ID/Boarding Pass has been checked and verified.
Passenger 409's ID/Boarding Pass has been checked and verified.
Passenger 429 arrives at the Airport Security at 77.00.
Passenger 405 enters Personal Scanner at 77.00.
Passenger 406 enters Personal Scanner at 77.00.
Passenger 409 enters Personal Scanner at 77.00.
Passenger 430 arrives at the Airport Security at 77.00.
Passenger 431 arrives at the Airport Security at 77.00.
Passenger 410 enters ID Check Station at 77.00.
Passenger 411 enters ID Check Station at 77.00.
Passenger 412 enters ID Check Station at 77.00.
Passenger 432 arrives at the Airport Security at 77.00.
Passenger 433 arrives at the Airport Security at 77.00.
Passenger 434 arrives at the Airport Security at 77.00.
Passenger 405 completes perso

Passenger 622 leaves Airport Security at 115.83.
Passenger 623 completes personal scanning process.
Passenger 623 leaves Airport Security at 115.86.
Passenger 621 completes personal scanning process.
Passenger 621 leaves Airport Security at 115.89.
Passenger 615 completes personal scanning process.
Passenger 615 leaves Airport Security at 115.90.
Passenger 614 completes personal scanning process.
Passenger 614 leaves Airport Security at 115.93.
Passenger 613 completes personal scanning process.
Passenger 613 leaves Airport Security at 115.96.
Passenger 720 arrives at the Airport Security at 116.00.
Passenger 624's ID/Boarding Pass has been checked and verified.
Passenger 626's ID/Boarding Pass has been checked and verified.
Passenger 721 arrives at the Airport Security at 116.00.
Passenger 624 enters Personal Scanner at 116.00.
Passenger 626 enters Personal Scanner at 116.00.
Passenger 629 enters ID Check Station at 116.00.
Passenger 630 enters ID Check Station at 116.00.
Passenger 629

Passenger 728's ID/Boarding Pass has been checked and verified.
Passenger 722 enters Personal Scanner at 136.00.
Passenger 829 arrives at the Airport Security at 136.00.
Passenger 728 enters Personal Scanner at 136.00.
Passenger 830 arrives at the Airport Security at 136.00.
Passenger 729 enters ID Check Station at 136.00.
Passenger 730 enters ID Check Station at 136.00.
Passenger 831 arrives at the Airport Security at 136.00.
Passenger 729's ID/Boarding Pass has been checked and verified.
Passenger 730's ID/Boarding Pass has been checked and verified.
Passenger 832 arrives at the Airport Security at 136.00.
Passenger 729 enters Personal Scanner at 136.00.
Passenger 730 enters Personal Scanner at 136.00.
Passenger 833 arrives at the Airport Security at 136.00.
Passenger 731 enters ID Check Station at 136.00.
Passenger 732 enters ID Check Station at 136.00.
Passenger 730 completes personal scanning process.
Passenger 730 leaves Airport Security at 136.51.
Passenger 722 completes persona

Passenger 887 enters Personal Scanner at 166.00.
Passenger 889 enters Personal Scanner at 166.00.
Passenger 892 enters Personal Scanner at 166.00.
Passenger 1006 arrives at the Airport Security at 166.00.
Passenger 1007 arrives at the Airport Security at 166.00.
Passenger 893 enters ID Check Station at 166.00.
Passenger 894 enters ID Check Station at 166.00.
Passenger 895 enters ID Check Station at 166.00.
Passenger 1008 arrives at the Airport Security at 166.00.
Passenger 893's ID/Boarding Pass has been checked and verified.
Passenger 894's ID/Boarding Pass has been checked and verified.
Passenger 1009 arrives at the Airport Security at 166.00.
Passenger 893 enters Personal Scanner at 166.00.
Passenger 894 enters Personal Scanner at 166.00.
Passenger 1010 arrives at the Airport Security at 166.00.
Passenger 1011 arrives at the Airport Security at 166.00.
Passenger 896 enters ID Check Station at 166.00.
Passenger 897 enters ID Check Station at 166.00.
Passenger 1012 arrives at the Airp

Passenger 1168 arrives at the Airport Security at 211.00.
Passenger 1147 enters ID Check Station at 211.00.
Passenger 1148 enters ID Check Station at 211.00.
Passenger 1149 enters ID Check Station at 211.00.
Passenger 1150 enters ID Check Station at 211.00.
Passenger 1148's ID/Boarding Pass has been checked and verified.
Passenger 1149's ID/Boarding Pass has been checked and verified.
Passenger 1148 enters Personal Scanner at 211.00.
Passenger 1149 enters Personal Scanner at 211.00.
Passenger 1151 enters ID Check Station at 211.00.
Passenger 1152 enters ID Check Station at 211.00.
Passenger 1152's ID/Boarding Pass has been checked and verified.
Passenger 1152 enters Personal Scanner at 211.00.
Passenger 1153 enters ID Check Station at 211.00.
Passenger 1145 completes personal scanning process.
Passenger 1145 leaves Airport Security at 211.51.
Passenger 1152 completes personal scanning process.
Passenger 1152 leaves Airport Security at 211.57.
Passenger 1146 completes personal scanning 

In [8]:
np.mean(wait_times)

12.896051817269358

In [10]:
# In order to keep average wait times below 15 minutes for arrivals of 5 passengers per minute, the simulation
# above determines one possible combination is 4 ID Check Stations and 2 Personal Check Scanners. However,
# the combinations can vary. It is also worth noting that adding ID Checking Stations seems to reduce wait times
# a greater degree more than personal scanners.